# Procesamiento de datos con Pandas en el subpaquete `teii.finance`

## Constructor de la clase derivada `TimesSeriesFinanceClient`

In [1]:
import datetime
import json
import numpy as np
import pandas as pd
import requests
from importlib import resources

In [2]:
pd.__version__

'1.3.5'

In [3]:
_data_field2name_type = {
            "1. open":                  ("open",     "float"),
            "2. high":                  ("high",     "float"),
            "3. low":                   ("low",      "float"),
            "4. close":                 ("close",    "float"),
            "5. adjusted close":        ("aclose",   "float"),
            "6. volume":                ("volume",   "int"),
            "7. dividend amount":       ("dividend", "float"),
        }

In [4]:
# Usamos IBM porque AMZN no repartió dividendos
query = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol=IBM&outputsize=full&apikey=MY_ALPHA_VANTAGE_API_KEY&data_type=json'

In [5]:
download_data = True   # Poner a True si queremos descargarnos los datos de la web

In [6]:
if download_data:
    # Realizamos la consulta
    response = requests.get(query)
    # print(response)
    # Extraemos los datos de la respuesta
    json_data_downloaded = response.json()
    # print(json.dumps(json_data_downloaded, indent=2))
    with open(f"TIME_SERIES_WEEKLY_ADJUSTED.IBM.json", 'w') as outfile:
            outfile.write(json.dumps(json_data_downloaded, indent=4))
else:
    with resources.open_text('teii.finance.data', 'TIME_SERIES_WEEKLY_ADJUSTED.IBM.json') as json_fid:
        json_data_downloaded = json.load(json_fid)

In [7]:
json_metadata = json_data_downloaded['Meta Data']
print(json.dumps(json_metadata, indent=2))

{
  "1. Information": "Weekly Adjusted Prices and Volumes",
  "2. Symbol": "IBM",
  "3. Last Refreshed": "2022-04-05",
  "4. Time Zone": "US/Eastern"
}


In [8]:
json_data = json_data_downloaded['Weekly Adjusted Time Series']
print(json.dumps(json_data, indent=2))

{
  "2022-04-05": {
    "1. open": "130.2600",
    "2. high": "131.2300",
    "3. low": "128.6600",
    "4. close": "128.8900",
    "5. adjusted close": "128.8900",
    "6. volume": "5913338",
    "7. dividend amount": "0.0000"
  },
  "2022-04-01": {
    "1. open": "130.8200",
    "2. high": "133.0800",
    "3. low": "128.0600",
    "4. close": "130.1500",
    "5. adjusted close": "130.1500",
    "6. volume": "19183786",
    "7. dividend amount": "0.0000"
  },
  "2022-03-25": {
    "1. open": "129.0000",
    "2. high": "131.4000",
    "3. low": "127.4000",
    "4. close": "131.3500",
    "5. adjusted close": "131.3500",
    "6. volume": "15440952",
    "7. dividend amount": "0.0000"
  },
  "2022-03-18": {
    "1. open": "124.4500",
    "2. high": "128.9300",
    "3. low": "122.6850",
    "4. close": "128.7600",
    "5. adjusted close": "128.7600",
    "6. volume": "22675632",
    "7. dividend amount": "0.0000"
  },
  "2022-03-11": {
    "1. open": "126.4700",
    "2. high": "128.3450",

In [9]:
# Build Panda's data frame
data_frame = pd.DataFrame.from_dict(json_data, orient='index', dtype=float)

In [11]:
data_frame.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount
2022-04-05,130.26,131.230,128.660,128.89,128.89,5913338.0,0.0
2022-04-01,130.82,133.080,128.060,130.15,130.15,19183786.0,0.0
2022-03-25,129.00,131.400,127.400,131.35,131.35,15440952.0,0.0
2022-03-18,124.45,128.930,122.685,128.76,128.76,22675632.0,0.0
2022-03-11,126.47,128.345,123.125,123.96,123.96,23784887.0,0.0


In [ ]:
# Diccionario con las conversiones de nombre de las columnas
{key: name_type[0] for key, name_type in _data_field2name_type.items()}

In [13]:
data_frame.rename?

In [14]:
# Rename data fields
data_frame = data_frame.rename(columns={key: name_type[0] for key, name_type in _data_field2name_type.items()})

In [15]:
data_frame.head()

,open,high,low,close,aclose,volume,dividend
2022-04-05,130.26,131.230,128.660,128.89,128.89,5913338.0,0.0
2022-04-01,130.82,133.080,128.060,130.15,130.15,19183786.0,0.0
2022-03-25,129.00,131.400,127.400,131.35,131.35,15440952.0,0.0
2022-03-18,124.45,128.930,122.685,128.76,128.76,22675632.0,0.0
2022-03-11,126.47,128.345,123.125,123.96,123.96,23784887.0,0.0


In [16]:
# Diccionario con las conversiones de tipos de las columnas
{name_type[0]: name_type[1] for key, name_type in _data_field2name_type.items()}

{'open': 'float',
 'high': 'float',
 'low': 'float',
 'close': 'float',
 'aclose': 'float',
 'volume': 'int',
 'dividend': 'float'}

In [18]:
# Set data field types
data_frame = data_frame.astype(dtype={name_type[0]: name_type[1] for key, name_type in _data_field2name_type.items()})

data_frame.dtypes

open        float64
high        float64
low         float64
close       float64
aclose      float64
volume        int64
dividend    float64
dtype: object

In [19]:
data_frame.astype?

In [20]:
# Set index type to datetime
data_frame.index = data_frame.index.astype("datetime64[ns]")

data_frame.index

DatetimeIndex(['2022-04-05', '2022-04-01', '2022-03-25', '2022-03-18',
               '2022-03-11', '2022-03-04', '2022-02-25', '2022-02-18',
               '2022-02-11', '2022-02-04',
               ...
               '2000-01-14', '2000-01-07', '1999-12-31', '1999-12-23',
               '1999-12-17', '1999-12-10', '1999-12-03', '1999-11-26',
               '1999-11-19', '1999-11-12'],
              dtype='datetime64[ns]', length=1170, freq=None)

In [21]:
# Sort data
data_frame = data_frame.sort_index(ascending=True)

data_frame.head()

,open,high,low,close,aclose,volume,dividend
1999-11-12,90.50,97.50,90.50,95.87,54.8757,43569700,0.12
1999-11-19,96.00,105.12,92.62,103.94,59.4949,61550800,0.00
1999-11-26,105.50,109.87,101.81,105.00,60.1017,37165600,0.00
1999-12-03,104.94,112.87,102.12,111.87,64.0340,37670000,0.00
1999-12-10,113.00,122.12,107.56,109.00,62.3912,58626000,0.00


A partir de aquí ya tenemos en `data_frame` los datos con los que vamos a trabajar, así que podemos usar este notebook para hacer las pruebas necesarias para extraer lo que nos piden en los métodos de `TimesSeriesFinanceClient()`.

## Solución a `weekly_volume()`

## Solución a `yearly_dividends()`

## Solución a `yearly_dividends_per_quarter()`

## Solución a `highest_weekly_variation()`

## Solución a `highest_monthly_mean_variation()`